Librerias

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Cargando datos

In [39]:
# Dataframe preprocesado
df = pd.read_csv('data/df.txt', sep='\t', encoding='latin-1')
df.head(20)

,asegurado_id,genero,ciudad,cancer,epoc,diabetes,hipertension,enf_cardiovascular,reclamacion,eventos,valor_pagado,estado_poliza,segmento_edad,diagnostico,tiempo_poliza
0,11885152,0,Bogota,0,0,0,0,0,CONSULTA EXTERNA,1,4.395772e+05,Poliza activa,Adultez,Diagnostico pendiente,24
1,763957,0,Medellin,0,0,0,0,0,LABORATORIO CLINICO,1,4.231473e+05,Poliza activa,Adultez,Diagnostico pendiente,24
2,763957,0,Medellin,0,0,0,0,0,EXAMENES DE DIAGNOSTICO,1,4.581114e+05,Poliza activa,Adultez,Diagnostico pendiente,24
3,763957,0,Medellin,0,0,0,0,0,EXAMENES DE DIAGNOSTICO,1,4.074159e+05,Poliza activa,Adultez,Factores que influyen en el estado de salud (S...,24
4,763957,0,Medellin,0,0,0,0,0,FISIOTERAPIA ILIMITADA,5,4.990100e+05,Poliza activa,Adultez,Diagnostico pendiente,24
5,763957,0,Medellin,0,0,0,0,0,EXAMENES DE DIAGNOSTICO,1,4.341680e+05,Poliza activa,Adultez,Factores que influyen en el estado de salud (S...,24
6,763957,0,Medellin,0,0,0,0,0,LABORATORIO CLINICO,2,4.869386e+05,Poliza activa,Adultez,Diagnostico pendiente,24
7,763957,0,Medellin,0,0,0,0,0,LABORATORIO CLINICO,4,5.434570e+05,Poliza activa,Adultez,Diagnostico pendiente,24
8,763957,0,Medellin,0,0,0,0,0,CONSULTA DE URGENCIAS,1,5.192157e+05,Poliza activa,Adultez,Diagnostico pendiente,24
9,763957,0,Medellin,0,0,0,0,0,CONSULTA EXTERNA,1,4.256361e+05,Poliza activa,Adultez,Diagnostico pendiente,24


In [40]:
# Calcular el valor promedio por evento
df['valor_pagado_promedio'] = df['valor_pagado'] / df['eventos']

# Eliminar las variables 'valor_pagado' y 'eventos'
df.drop(['valor_pagado', 'eventos','asegurado_id'], axis=1, inplace=True)

# Muestra las primeras filas para verificar los cambios
df.head(5)

,genero,ciudad,cancer,epoc,diabetes,hipertension,enf_cardiovascular,reclamacion,estado_poliza,segmento_edad,diagnostico,tiempo_poliza,valor_pagado_promedio
0,0,Bogota,0,0,0,0,0,CONSULTA EXTERNA,Poliza activa,Adultez,Diagnostico pendiente,24,439577.171642
1,0,Medellin,0,0,0,0,0,LABORATORIO CLINICO,Poliza activa,Adultez,Diagnostico pendiente,24,423147.287954
2,0,Medellin,0,0,0,0,0,EXAMENES DE DIAGNOSTICO,Poliza activa,Adultez,Diagnostico pendiente,24,458111.418782
3,0,Medellin,0,0,0,0,0,EXAMENES DE DIAGNOSTICO,Poliza activa,Adultez,Factores que influyen en el estado de salud (S...,24,407415.923794
4,0,Medellin,0,0,0,0,0,FISIOTERAPIA ILIMITADA,Poliza activa,Adultez,Diagnostico pendiente,24,99802.006128


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3778956 entries, 0 to 3778955
Data columns (total 13 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   genero                 int64  
 1   ciudad                 object 
 2   cancer                 int64  
 3   epoc                   int64  
 4   diabetes               int64  
 5   hipertension           int64  
 6   enf_cardiovascular     int64  
 7   reclamacion            object 
 8   estado_poliza          object 
 9   segmento_edad          object 
 10  diagnostico            object 
 11  tiempo_poliza          int64  
 12  valor_pagado_promedio  float64
dtypes: float64(1), int64(7), object(5)
memory usage: 374.8+ MB


In [42]:
# Convertir columnas binarias al tipo int64
df['genero'] = df['genero'].astype('int64')
df['cancer'] = df['cancer'].astype('int64')
df['epoc'] = df['epoc'].astype('int64')
df['diabetes'] = df['diabetes'].astype('int64')
df['hipertension'] = df['hipertension'].astype('int64')
df['enf_cardiovascular'] = df['enf_cardiovascular'].astype('int64')
df['tiempo_poliza'] = df['tiempo_poliza'].astype('int64')

# Convertir las variables ciudad y segmento_edad a categóricas
df['ciudad'] = df['ciudad'].astype('category')
df['segmento_edad'] = df['segmento_edad'].astype('category')
df['diagnostico'] = df['diagnostico'].astype('category')
df['reclamacion'] = df['diagnostico'].astype('category')
df['estado_poliza'] = df['estado_poliza'].astype('category')

In [43]:
# Selección de características y target
X = df.drop(columns=['valor_pagado_promedio', 'reclamacion', 'diagnostico'])
y = df['valor_pagado_promedio']

# Identificación de variables numéricas y categóricas
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Preprocesamiento de las características
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Definición de los modelos
models = {
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(n_estimators=100),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100)
}


In [44]:
# Supongamos que tienes un DataFrame llamado 'datos' que contiene la variable 'valor_pagado_promedio'
# Puedes cambiar 'datos' por el nombre de tu DataFrame y 'valor_pagado_promedio' por el nombre de tu variable

# Definir los límites de las categorías
categorias = [0, 500000, 3000000, 10000000, 100000000, 2000000000]

# Definir los nombres de las categorías
nombres_categorias = [0, 1, 2, 3, 4]
#nombres_categorias = ['Muy bajo', 'Bajo', 'Medio', 'Alto', 'Muy alto']

# Recategorizar la variable
df['categoria_valor_pagado'] = pd.cut(df['valor_pagado_promedio'], bins=categorias, labels=nombres_categorias, right=False)

# Mostrar los primeros registros del DataFrame con la nueva categoría
print(df.head())

   genero    ciudad  cancer  epoc  diabetes  hipertension  enf_cardiovascular  \
0       0    Bogota       0     0         0             0                   0   
1       0  Medellin       0     0         0             0                   0   
2       0  Medellin       0     0         0             0                   0   
3       0  Medellin       0     0         0             0                   0   
4       0  Medellin       0     0         0             0                   0   

                                         reclamacion  estado_poliza  \
0                              Diagnostico pendiente  Poliza activa   
1                              Diagnostico pendiente  Poliza activa   
2                              Diagnostico pendiente  Poliza activa   
3  Factores que influyen en el estado de salud (S...  Poliza activa   
4                              Diagnostico pendiente  Poliza activa   

  segmento_edad                                        diagnostico  \
0       Adultez 

In [45]:
# Contar la cantidad de valores NaN en la columna 'valor_pagado_promedio'
nan_count = df['categoria_valor_pagado'].isna().sum()

# Imprimir la cantidad de valores NaN
print("Cantidad de valores NaN en 'valor_pagado_promedio_categorias':", nan_count)

Cantidad de valores NaN en 'valor_pagado_promedio_categorias': 3165


In [46]:
display(df['categoria_valor_pagado'].unique().tolist())

[0, 1, 3, nan, 2, 4]

In [47]:
df = df.dropna(subset=['categoria_valor_pagado'])

In [48]:
# Codificar variables categóricas
df_encoded = pd.get_dummies(df, columns=['ciudad', 'reclamacion', 'estado_poliza', 'segmento_edad', 'diagnostico'])

# Separar características y variable objetivo
X = df_encoded.drop(['valor_pagado_promedio'], axis=1)
y = df_encoded['valor_pagado_promedio']

In [49]:
# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [50]:
# Paso 1: Normalización de los datos de la variable objetivo
scaler = MinMaxScaler()
y_train_normalized = scaler.fit_transform(y_train.values.reshape(-1, 1))

# Paso 2: Entrenamiento del modelo de regresión
model = RandomForestRegressor(n_estimators=50, random_state=24)
model.fit(X_train_scaled, y_train_normalized)

# Evaluar el modelo (si lo deseas)
y_pred_normalized = model.predict(X_test_scaled)

# Paso 3: Invertir la transformación para obtener las predicciones en la escala original
y_pred = scaler.inverse_transform(y_pred_normalized.reshape(-1, 1))

c:\Users\Javier Burgos\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [51]:
# División de los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Evaluación de modelos
for name, model in models.items():
    # Crear pipelinee
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])
    
    # Validación cruzada
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_rmse = np.sqrt(-cv_scores)
    print(f"{name} - CV RMSE: {cv_rmse.mean():.2f} ± {cv_rmse.std():.2f}")
    
    # Entrenamiento y predicción
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    # Evaluación en el conjunto de prueba
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"{name} - Test RMSE: {rmse:.2f}, R^2: {r2:.2f}\n")

Decision Tree - CV RMSE: 2826847.45 ± 410651.56
Decision Tree - Test RMSE: 2815930.20, R^2: 0.00

Random Forest - CV RMSE: 2826850.26 ± 410651.83
Random Forest - Test RMSE: 2816097.64, R^2: 0.00

Gradient Boosting - CV RMSE: 2826850.90 ± 410779.44
Gradient Boosting - Test RMSE: 2815793.01, R^2: 0.00



In [ ]:
# Guardar el modelo con mejor desempeño
#best_model = models["Random Forest"]
#best_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                                ('model', best_model)])
#best_pipeline.fit(X_train, y_train)
#joblib.dump(best_pipeline, 'best_model.pkl')